<a href="https://colab.research.google.com/github/elenrol02/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/Elena_Rold%C3%A1n_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3 - ACTIVIDAD GUIADA 1 <br>

Elena Roldán Del Cerro <br>

URL: https://github.com/elenrol02/03MAIR-Algoritmos-de-optimizacion/tree/master/AG3 <br>

In [2]:
#Problema del agente viajero

import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f55bfddc240>)

In [3]:
#Libreria para cargar toda la informacion

!pip install tsplib95

    100% |████████████████████████████████| 1.6MB 12.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import tsplib95
import random
from math import e

problem = tsplib95.load_problem(file)


#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges()) 



In [0]:
#Devuelve el factorial de un numero
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
              
#Se genera una solucion aleatoria con comienzo en el nodo 0
#Cogemos todos los nodos desde 0 y se va añadiendo a la solución en cada ciclo un nuevo nodo de forma aleatoria quitando el cero y los nodos que ya aprecen en solucion
def crear_solucion(Nodos):
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)
                 
#Devuelve la distancia total de una trayectoria
#Hay que sumarle la distancia desde el punto final al punto inicial
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)                 



In [52]:
#Búsqueda aleatoria
#N es la condición de parada
def busqueda_aleatoria(problem, N):
  
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  #Se define una distancia inicial muy grande
  mejor_distancia = 10e100
  
  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)
    
    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion

sol=busqueda_aleatoria(problem,100)

Mejor solución: [0, 17, 9, 5, 26, 14, 11, 12, 18, 29, 1, 8, 19, 39, 38, 3, 30, 23, 28, 15, 16, 7, 4, 22, 34, 33, 25, 10, 32, 2, 13, 24, 21, 40, 31, 6, 41, 27, 36, 35, 20, 37]
Distancia     : 3923


In [53]:
#Búsqueda local
#Generación de vencindad: se basa en intercambiar dos nodos.
#Se incializa una variable para contemplar la solucion y las distancias y con un bucle anidado se van intercambiando parejas de nodos


def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] #Dividir la lista en 4 trozos
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

solucion=crear_solucion(Nodos)
print (solucion)

nueva_solucion=genera_vecina(solucion)
print (nueva_solucion)



[0, 9, 41, 25, 13, 19, 10, 2, 11, 29, 33, 28, 21, 36, 23, 24, 6, 35, 26, 14, 7, 1, 39, 3, 17, 40, 31, 12, 4, 18, 27, 32, 22, 16, 8, 34, 37, 5, 15, 20, 38, 30]
[0, 9, 41, 25, 13, 19, 10, 2, 11, 29, 33, 28, 21, 40, 23, 24, 6, 35, 26, 14, 7, 1, 39, 3, 17, 36, 31, 12, 4, 18, 27, 32, 22, 16, 8, 34, 37, 5, 15, 20, 38, 30]


In [59]:
def busqueda_local(problem,N):
  mejor_solucion=[]
  mejor_distancia = 10e100
  
  solucion_referencia=crear_solucion(Nodos)
  
  for i in range(N):
    vecina=genera_vecina(solucion_referencia)
    distancia_vecina=distancia_total(vecina,problem)
    
    if distancia_vecina<=mejor_distancia:
      mejor_solucion=vecina
      mejor_distancia=distancia_vecina
    solucion_referencia=vecina
    
  print ('La mejor solucion encontrada es' , end='')
  print (mejor_solucion)
  print ('con una distancia total de ' , end='')
  print (mejor_distancia)
  return mejor_solucion


busqueda_local(problem,100)

La mejor solucion encontrada es[0, 30, 38, 22, 24, 40, 35, 36, 37, 1, 4, 12, 11, 25, 10, 21, 39, 34, 33, 20, 9, 23, 41, 8, 29, 28, 2, 27, 3, 15, 16, 14, 19, 13, 18, 26, 5, 6, 7, 17, 31, 32]
con una distancia total de 2103


[0,
 30,
 38,
 22,
 24,
 40,
 35,
 36,
 37,
 1,
 4,
 12,
 11,
 25,
 10,
 21,
 39,
 34,
 33,
 20,
 9,
 23,
 41,
 8,
 29,
 28,
 2,
 27,
 3,
 15,
 16,
 14,
 19,
 13,
 18,
 26,
 5,
 6,
 7,
 17,
 31,
 32]

In [0]:
#Recocido simulado permite escapar de mínimos locales
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no sea uniforme sino entre las que estén más proximas
  #Crea cualquier solución vecina
  i = random.choice(range(1, len(solucion)))
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  return vecina

#Funcion que depende de la temperatura y de la distancia entre dos soluciones
def probabilidad (T,d):
  r=random.random()
  if(r <= (e**(-1*d)/(T*1.0))):
     return True
  else:
     return False
    
def bajar_temperatura(T):
  return T-1



In [73]:
def recocido_simulado(problem, temperatura):
  solucion_referencia=crear_solucion(Nodos)
  distancia_referencia=distancia_total(solucion_referencia,problem)
  
  mejor_solucion=[]
  mejor_distancia = 10e100
  
  #Criterio de parada
  while temperatura > 0:
    #Genera una solución vecina aleatoria
    vecina=genera_vecina_aleatorio(solucion_referencia)
    distancia_vecina=distancia_total(vecina,problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina <= mejor_distancia:
      mejor_solucion=vecina
      mejor_distancia=distancia_vecina
     
    #Si la distancia de la nueva vecina es menor que la distancia de referencia pasa a ser nuestra distancia de referencia, si esto no ocurre también se puede cambiar
    #la solución de referencia a una solución vecina que es peor pero con una probabilidad que hemos definido
    if distancia_vecina < distancia_referencia or probabilidad(temperatura,abs(distancia_referencia-distancia_vecina) ):
      solucion_referencia=vecina
      distancia_referencia=distancia_vecina
      
    temperatura=bajar_temperatura(temperatura)
    
  print ('La mejor solucion encontrada es' , end='')
  print (mejor_solucion)
  print ('con una distancia total de ' , end='')
  print (mejor_distancia)
  return mejor_solucion

sol=recocido_simulado(problem,10000)
    
    
      
      
    
  

La mejor solucion encontrada es[0, 1, 19, 13, 4, 2, 27, 3, 6, 14, 16, 15, 37, 7, 28, 30, 38, 22, 39, 21, 40, 24, 33, 35, 36, 31, 17, 5, 26, 18, 10, 41, 23, 9, 9, 8, 8, 25, 11, 12, 29, 34, 20, 32]
con una distancia total de 1843


In [0]:
#Colonia de hormigas: funciones auxiliares
def add_nodo(problem, H,T):
  #A partir de una solución h, añadir en cada paso un nodo más. Habría que hacerlo en base a la feromona t y en base a la distancia más cercana
  #de los nodos que me queden por recorrer
  Nodos=list(problem.get_nodes())
  return random.choice(list(set(range(1,len(Nodos)))-set(H)))

def incrementa_feromona(problem,T,H):
  #Recorrer trayectoria H y para cada una de las aristas, guardar una cantidad que es inversamente proporcional a la distancia que nos da esa solucion
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def evaporar_feromonas(T):
  #reduces las feromonas en 0.3
  T=[[max(T[i][j] - 0.3, 1) for i in range(len(Nodos))] for j in range (len(Nodos))]
  return T



In [0]:
#Colonia de hormigas
def hormigas(problem,N):
  Nodos = list(problem.get_nodes())
  Aristas = list(problem.get_edges()) 

  #Inicializa las aristas con una cantidad inicial de feromonas: 1
  T=[[1 for _ in range(len(Nodos))] for _ in range(len(Nodos))]
  
  #Se generan los agentes que serán estructuras de camino desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N):
    for i in range (len(Nodos)-1):
      
      #Elige el siguiente nodo
      nuevo_nodo=add_nodo(problem, Hormiga[h],T)
      Hormiga[h].append(nuevo_nodo)
      
      #Incrementa feromonas en esa arista
      T=incrementa_feromona(problem, T, Hormiga[h])
      
      #Evapora feromonas
      T=evaporar_feromonas(T)
      
  #Seleccionar el mejor agente
  mejor_solucion=[]
  mejor_distancia = 10e100
  for h in range(N):
    distancia_actual=distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion= Hormiga[h]
      mejor_distancia=distancia_actual
      
hormigas(problem,100)
  

In [76]:
T=[[1 for _ in range(len(Nodos))] for _ in range(len(Nodos))]

print (T)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [61]:
print (random.choice(range(1, len(solucion))))

37


In [64]:
print (solucion[24])

17
